# Tutorial for interactive histogramming
* imprt libraries
* generate random data source ABCD uniform
* create figure layout and histogram aray
* simple hstogramming
* histogramming with table
* histogramming with weights

In [ ]:
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import os
import sys
import pytest
from pandas import CategoricalDtype
from bokeh.io import output_notebook
output_notebook()
# import logging

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# generate random data source ABCD uniform
* anotating data with metadata
* e.g axisTitle:
   * "'A.AxisTitle': "A (cm)"
* here we will put link to github to see list of option

In [ ]:
df = pd.DataFrame(np.random.random_sample(size=(20000, 4)), columns=list('ABCD'))
initMetadata(df)
MARKERS = ['hex', 'circle_x', 'triangle','square']
markerFactor=factor_mark('DDC', MARKERS, ["A0","A1","A2","A3","A4"] )
colorFactor=factor_cmap('DDC', 'Category10_6', ["A0","A1","A2","A3","A4"] )

mapDDC={0:"A0",1:"A1",2:"A2",3:"A3",4:"A4"}
df.eval("Bool=A>0.5", inplace=True)
df.eval("BoolB=B>0.5", inplace=True)
df["AA"]=((df.A*10).round(0)).astype(CategoricalDtype(ordered=True))
df["CC"]=((df.C*5).round(0)).astype(int)
df["DD"]=((df.D*4).round(0)).astype(int)
df["DDC"]=((df.D*4).round(0)).astype(int).map(mapDDC)
df['errY']=df.A*0.02+0.02
df.head(10)
df.meta.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm)", 'C.AxisTitle': "C (s)",
                    'D.AxisTitle': "D (cm)", 'Bool.AxisTitle': "A>half"}


#  create figure layout and histogram aray
* tooltips
  * what will be shown on the mouse over
  * in case blow tooltips for varaible A, N, C and D
  * variables to bokeh is interpreted as string @is adress operator 
    * it is possible to get fuction on client but has to be intgrate in wrappers 
* Widget params:
  * [type, params]
      * range
      * slider
      * select
      * multiselect
      * currently we are using array of parameters - positional argument
      * in next version  dictionary and positional arguments .... 
* widgetLayoutDesc, figureLayoutDesc
  * array of the rows with widget indices
  * last argument are the options
     * link to the git - possible options
  * in bokeh widgtets can not be reused
  
* hitoArray
   * array of histogram description
   * convention as in histogramdd in numpy
      * name 
      * varaibles
      * nbins - sclar in array
      * ranges [max,min] 0 in case not specified minimum maximum of data points for current selection  used
      * qunatiles, sum_range, axis
         * parameters of agreagation function
   * columns in created histograms
      * 1D histogram:
          * bin_bottom, bin_center, bin_top - bin edges and centers
          * bin_count - (weighted) count
      * ND histogram (N>1):
          * bin_bottom_i, bin_center_i, bin_top_i - bin edges and centers on the histogram's i-th axis
          * bin_count - (weighted) count
      * ND histogram projection (can be accessed in bokehDraw by histoName_i where histoName is the histogram's name and i is the index of the projection axis):
          * bin_bottom_i, bin_center_i, bin_top_i - bin edges and centers on the histogram projection's i-th axis
          * entries - the (weighted) count of entries
          * mean - the (weighted) mean value of the i-th axis in the bin
          * std - the (weighted) mean value of the i-th axis in the bin
          * quantile_i - the i-th quantile specified in the quantiles list in histoArray
          

In [ ]:
tooltips = [("VarA", "(@A)"), ("B", "(@B)"), ("C (s)", "(@C)"), ("VarD (cm)", "(@D)"),  ("VarD (1/8cm)", "(8*@D)")]

widgetParams=[
    ['range', ['A']],
    ['range', ['B', 0, 1, 0.1, 0, 1]],
    ['range', ['C'], {'type': 'minmax'}],
    ['range', ['D'], {'type': 'sigma', 'bins': 10, 'sigma': 3}],
    ['multiSelect',["BoolB"]],
]
widgetLayoutDesc=[[0, 1, 2], [3, 4], {'sizing_mode': 'scale_width'}]

# Simple drawing without table and with original data superimposed
* A, B and C from uniform distribution
* histograms of A, B
* histogram of (A+B)/2 with weights A*C

In [ ]:
def testBokehClientHistogram():
    output_file("test_BokehClientHistogram.html")
    histoArray = [
        {"name": "histoA", "variables": ["A"], "nbins":20, "quantiles": [.05, .5, .95], "sum_range": [[.25, .75], [.4, .6]]},
        {"name": "histoB", "variables": ["B"], "nbins":20, "range": [0, 1]},
        {"name": "histoTransform", "variables": ["(A+B)/2"], "nbins": 20, "weights": "A*C"},
    ]
    figureArray = [
        #   ['A'], ['C-A'], {"color": "red", "size": 7, "colorZvar":"C", "filter": "A<0.5"}],
        [['A'], ['histoA', '(A*A-C*C)*100'], {"size": 2, "colorZvar": "A", "errY": "errY", "errX": "0.01"}],
        [['(A+B)/2'], ['histoTransform', '(C+A)*200', '(C-A)*200']],
        [['B'], ['histoB', '(C+B)*10', '(C-B)*10'], {"size": 7, "colorZvar": "C", "errY": "errY",
                                                    "rescaleColorMapper": True}]
    ]
    figureLayoutDesc=[
        [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
    ]
    
    xxx=bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                              widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=3000, histogramArray=histoArray)
testBokehClientHistogram()

# Example with table aggregation

In [ ]:
def testBokehClientHistogramRowwiseTable():
    output_file("test_BokehClientHistogramRowwiseTable.html")
    histoArray = [
        {"name": "histoA", "variables": ["A"], "nbins":20, "quantiles": [.05, .5, .95], "sum_range": [[.25, .75], [.4, .6]]},
        {"name": "histoB", "variables": ["B"], "nbins":20, "range": [0, 1]},
        {"name": "histoTransform", "variables": ["(A+B)/2"], "nbins": 20, "weights": "A*C"},
    ]
    figureArray = [
        [['A'], ['histoA']],
        [['B'], ['histoB'], {"show_histogram_error": True}],
        [['(A+B)/2'], ['histoTransform'], {"yAxisTitle": "sum A*C"}],
        ["tableHisto", {"rowwise": True}]
    ]
    figureLayoutDesc=[
        [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        [3],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
    ]
    xxx = bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                                widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", histogramArray=histoArray)
testBokehClientHistogramRowwiseTable()

# Example with 2D histogram and projections
*  Uniform distribution in varable A,B
* showing histogram of A vs (A+B)/2, mean, median, RMS, quantiles of (A+B/2) as a function of A

In [ ]:
def testBokehClientHistogramProfileA():
    output_file("test_BokehClientHistogramProfileA.html")
    histoArray = [
        {"name": "histoAB", "variables": ["A", "(A+B)/2"], "nbins": [20, 20], "axis": [0, 1], "quantiles": [.1, .5, .9]}
    ]
    figureArray = [
        [['histoAB_1.bin_center_0'], ['histoAB_1.quantile_0', 'histoAB_1.quantile_1', 'histoAB_1.quantile_2']],
        [['histoAB_1.bin_center_0'], ['histoAB_1.quantile_1', 'histoAB_1.mean']],
        [['A'], ['histoAB'], {"yAxisTitle": "(A+B)/2"}],
        [['histoAB_1.bin_center_0'], ['histoAB_1.std']],
        ["tableHisto", {"rowwise": False}]
    ]
    figureLayoutDesc=[
        [0, 1,  {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
        [2, 3, {'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
        [4, {'plot_height': 40}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2, "size": 5}
    ]
    xxx = bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                                widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", histogramArray=histoArray)
testBokehClientHistogramProfileA()

# Example with 2D histogram and projections
*  Uniform distribution in varable A,B
* showing histogram of A vs (A+B)/2, mean, median, RMS, quantiles of A as a function of (A+B)/2

In [ ]:
def testBokehClientHistogramProfileB():
    output_file("test_BokehClientHistogramProfileB.html")
    histoArray = [
        {"name": "histoAB", "variables": ["A", "(A+B)/2"], "nbins": [20, 20], "weights": "D", "axis": [0, 1], "quantiles": [.1, .5, .9]}
    ]
    figureArray = [
        [['histoAB_0.quantile_0', 'histoAB_0.quantile_1', 'histoAB_0.quantile_2'], ['histoAB_0.bin_center_1']],
        [['histoAB_0.quantile_1', 'histoAB_0.mean'], ['histoAB_0.bin_center_1']],
        [['A'], ['histoAB'], {"yAxisTitle": "(A+B)/2"}],
        [['histoAB_0.std'], ['histoAB_0.bin_center_1']],
        ["tableHisto", {"rowwise": False}]
    ]
    figureLayoutDesc=[
        [0, 1,  {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
        [2, 3, {'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
        [4, {'plot_height': 40}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2, "size": 5}
    ]
    xxx = bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                                widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", histogramArray=histoArray)
testBokehClientHistogramProfileB()

# Example with projections of a 3D histogram into 2D
*  Uniform distribution in varable A,B,C, weighted 3D histogram of (A+C)/2, B, C with D as weights
* showing mean and rms in (A+C)/2 as function of B with C bin center as a color

In [ ]:
def testBokehClientHistogram3d():
    output_file("test_BokehClientHistogram.html")
    histoArray = [
        {"name": "histoABC", "variables": ["(A+C)/2", "B", "C"], "nbins": [8, 10, 12], "weights": "D", "axis": [0], "sum_range": [[.25, .75]]},
    ]
    figureArray = [
        [['histoABC_0.bin_center_1'], ['histoABC_0.mean'], {"colorZvar": "histoABC_0.bin_center_2", "size": 7}],
        #[['histoABC_0.bin_center_1'], ['histoABC_0.sum_0'], {"colorZvar": "histoABC_0.bin_center_2", "size": 7}],
        [['histoABC_0.bin_center_1'], ['histoABC_0.std'], {"colorZvar": "histoABC_0.bin_center_2", "size": 7}]
    ]
    figureLayoutDesc=[
        [0, 1, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
    ]
    
    xxx=bokehDrawSA.fromArray(df, "A>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                              widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", nPointRender=3000, histogramArray=histoArray)
testBokehClientHistogram3d()